# Calculate SI from NT Mosaic Reflectance Data

Extracts mean band values (6 bands) and calculates multiple common ratio and band indexes from merged_slats_field_agb_nt_mosaic_clean_start.csv
The following conditions apply:

 - env = base
 - Run following Merged Tile AGB Zonal Stats Colation.ipynb


In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
pd.set_option('chained_assignment',None)
from __future__ import division
#pd.set_option('precision', 8)

In [2]:
drive = "F"
date = "20230109"

In [3]:
# this note book will take in the combined training and validation data which has been combined and produces the 
# 
# ratios and vegetation indices for Landsat-8 (use the one written for the sentinel-2) and converts them to int32 bit data 
# 
# and reduces the significant figures to 8 decimal places. Comment out the relevent input csv file at the start and finish
#

In [4]:
# read in the zonal stats results
df = pd.read_csv(r"{0}:\cdu\data\zonal_stats\output\{1}\dja_dbi_dim_dis_dka_stc_h99a2_fpca2_dry_clean_df.csv".format(drive, date), header=0)
print (list(df))
print (df.shape)

['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'geometry', 'b1_dja_min', 'b1_dja_max', 'b1_dja_mean', 'b1_dja_med', 'b1_dja_std', 'b1_dja_p25', 'b1_dja_p50', 'b1_dja_p75', 'b1_dja_p95', 'b1_dja_p99', 'b1_dbi_min', 'b1_dbi_max', 'b1_dbi_mean', 'b1_dbi_med', 'b1_dbi_std', 'b1_dbi_p25', 'b1_dbi_p50', 'b1_dbi_p75', 'b1_dbi_p95', 'b1_dbi_p99', 'b2_dbi_min', 'b2_dbi_max', 'b2_dbi_mean', 'b2_dbi_med', 'b2_dbi_std', 'b2_dbi_p25', 'b2_dbi_p50', 'b2_dbi_p75', 'b2_dbi_p95', 'b2_dbi_p99', 'b3_dbi_min', 'b3_dbi_max', 'b3_dbi_mean', 'b3_dbi_med', 'b3_dbi_std', 'b3_dbi_p25', 'b3_dbi_p50', 'b3_dbi_p75', 'b3_dbi_p95', 'b3_dbi_p99', 'b4_dbi_min', 'b4_dbi_max', 'b4_dbi_mean', 'b4_dbi_med', 'b4_dbi_std', 'b4_dbi_p25', 'b4_dbi_p50', 'b4_dbi_p75', 'b4_dbi_p95', 'b4_dbi_p99', 'b5

In [5]:
a = df.copy()

In [6]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Columns: 161 entries, uid to fpca2_imdate
dtypes: float64(156), int64(2), object(3)
memory usage: 210.2+ KB


In [7]:
a = a[["site", 'bio_agb_kg1ha', 'b1_dbi_mean', 'b2_dbi_mean', 'b3_dbi_mean', 'b4_dbi_mean', 'b5_dbi_mean', 'b6_dbi_mean']]

In [8]:
a.columns = ["site", "total_agb", "psB1a", "psB2a", "psB3a", "psB4a", "psB5a", "psB6a", ]

In [9]:
a.dropna(inplace=True)

In [10]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   site       167 non-null    object 
 1   total_agb  167 non-null    float64
 2   psB1a      167 non-null    float64
 3   psB2a      167 non-null    float64
 4   psB3a      167 non-null    float64
 5   psB4a      167 non-null    float64
 6   psB5a      167 non-null    float64
 7   psB6a      167 non-null    float64
dtypes: float64(7), object(1)
memory usage: 11.7+ KB


In [11]:
# read in the zonal stats results
#a = pd.read_csv("combined_val_data_l8.csv", header=0)
#print (list(a))

In [12]:
# converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# to floating point for analysis of the vegetation index, blue band not used

blue =((a['psB1a'].astype('int16')*0.0001)+0.0)

green =((a['psB2a'].astype('int16')*0.0001)+0.0)
red = ((a['psB3a'].astype('int16')*0.0001)+0.0)
nir = ((a['psB4a'].astype('int16')*0.0001)+0.0)
swir1 =((a['psB5a'].astype('int16')*0.0001)+0.0)
swir2 =((a['psB6a'].astype('int16')*0.0001)+0.0)

In [13]:
# calculate the band ratios

a['ratio32fa'] = (a['psB3a'] / a['psB2a'])
a['ratio42fa'] = (a['psB4a'] / a['psB2a'])
a['ratio43fa'] = (a['psB4a'] / a['psB3a'])
a['ratio52fa'] = (a['psB5a'] / a['psB2a'])
a['ratio53fa'] = (a['psB5a'] / a['psB3a'])
a['ratio54fa'] = (a['psB5a'] / a['psB4a'])
a['ratio62fa'] = (a['psB6a'] / a['psB2a'])
a['ratio63fa'] = (a['psB6a'] / a['psB3a'])
a['ratio64fa'] = (a['psB6a'] / a['psB4a'])
a['ratio65fa'] = (a['psB6a'] / a['psB5a'])

# calculate the band ratios and convert them to int32 bit at 7 decimal places

a['ratio32a'] = np.int32(np.around(a['ratio32fa'] * 10 ** 7))

a['ratio42a'] = np.int32(np.around(a['ratio42fa'] * 10 ** 7))

a['ratio43a'] = np.int32(np.around(a['ratio43fa'] * 10 ** 7))

a['ratio52a'] = np.int32(np.around(a['ratio52fa'] * 10 ** 7))

a['ratio53a'] = np.int32(np.around(a['ratio53fa'] * 10 ** 7))

a['ratio54a'] = np.int32(np.around(a['ratio54fa'] * 10 ** 7))

a['ratio62a'] = np.int32(np.around(a['ratio62fa'] * 10 ** 7))

a['ratio63a'] = np.int32(np.around(a['ratio63fa'] * 10 ** 7))

a['ratio64a'] = np.int32(np.around(a['ratio64fa'] * 10 ** 7))

a['ratio65a'] = np.int32(np.around(a['ratio65fa'] * 10 ** 7))

In [14]:
# calculate the vegetation indices
a['GSAVIfa'] = ((nir-green)/(nir+green+0.5))*(1.5)
a['GSAVIa'] = np.int32(np.around(a['GSAVIfa'] * 10 ** 7))

a['GNDVIfa'] = (nir-green)/(nir+green)
a['GNDVIa'] = np.int32(np.around(a['GNDVIfa'] * 10 ** 7))

a['CVIfa']= (nir/green)*(red/green)
a['CVIa'] = np.int32(np.around(a['CVIfa'] * 10 ** 7))

a['NDGIfa']= (green-red)/(green+red)
a['NDGIa'] = np.int32(np.around(a['NDGIfa'] * 10 ** 7))

a['RIfa']= (red-green)/(red+green)
a['RIa'] = np.int32(np.around(a['RIfa'] * 10 ** 7))

a['NBRfa']= (nir-swir2)/(nir+swir2)
a['NBRa'] = np.int32(np.around(a['NBRfa'] * 10 ** 7))

a['NDIIfa']= (nir-swir1)/(nir+swir1)
a['NDIIa'] = np.int32(np.around(a['NDIIfa'] * 10 ** 7))

a['GDVIfa']= (nir-green)
a['GDVIa'] = np.int32(np.around(a['GDVIfa'] * 10 ** 7))

a['MSAVIfa'] = (2 * nir + 1 - np.sqrt((np.power(((2*nir)+1) , 2)) - (8 * (nir-red))))/2
a['MSAVIa'] = np.int32(np.around(a['MSAVIfa'] * 10 ** 7))

a['DVIfa'] = (nir-red)
a['DVIa'] = np.int32(np.around(a['DVIfa'] * 10 ** 7))

a['SAVIfa']= ((nir-red)/(nir+red+0.5))*(1+0.5)
a['SAVIa'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

a['NDVIfa']= (nir-red)/(nir+red)
a['NDVIa'] = np.int32(np.around(a['NDVIfa'] * 10 ** 7))

a['MSRfa']= (((nir/red)-1)/((np.sqrt(nir/red))+1))
a['MSRa'] = np.int32(np.around(a['MSRfa'] * 10 ** 7))

In [15]:
a

,site,total_agb,psB1a,psB2a,psB3a,psB4a,psB5a,psB6a,ratio32fa,ratio42fa,...,MSAVIfa,MSAVIa,DVIfa,DVIa,SAVIfa,SAVIa,NDVIfa,NDVIa,MSRfa,MSRa
0,nt001,0.000000,426.875,728.1250,1047.6875,2381.4375,3321.8125,2027.0000,1.438884,3.270644,...,0.210851,2108509,0.1334,1334000,0.237423,2374229,0.389148,3891482,0.508017,5080174
1,barkely01,0.000000,983.200,1306.7500,1675.6500,2406.9000,3099.0500,2337.3500,1.282303,1.841898,...,0.106338,1063379,0.0731,731000,0.120747,1207466,0.179123,1791228,0.198507,1985065
2,barkely02,0.000000,977.240,1298.5600,1779.8400,2618.6800,3180.0800,2249.6400,1.370626,2.016603,...,0.119508,1195078,0.0839,839000,0.133926,1339257,0.190812,1908119,0.213101,2131007
3,barkely04,0.000000,1037.240,1415.8000,1912.7600,2589.9200,3153.7200,2133.6800,1.351010,1.829298,...,0.095176,951763,0.0677,677000,0.106883,1068835,0.150411,1504110,0.163649,1636492
4,barkely03,0.000000,907.160,1265.6800,1705.8000,2443.2400,3453.8000,2444.8800,1.347734,1.930377,...,0.106819,1068186,0.0738,738000,0.121010,1210101,0.177917,1779171,0.197015,1970149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,ntadac0002,20805.217611,244.875,430.1250,490.4375,2258.0000,1752.0000,881.2500,1.140221,5.249637,...,0.309644,3096442,0.1768,1768000,0.342282,3422819,0.643377,6433770,1.146663,11466633
163,ntaarp0001,30472.446667,244.550,447.7500,603.4500,2193.7500,1750.6500,936.9000,1.347739,4.899497,...,0.272766,2727661,0.1590,1590000,0.305926,3059261,0.568670,5686695,0.907044,9070438
164,ntaarp0002,24414.130222,366.600,628.8400,955.6000,2438.0000,2332.2000,1427.2800,1.519623,3.876980,...,0.237205,2372051,0.1483,1483000,0.265042,2650423,0.437076,4370763,0.597773,5977733
165,ntaarp0003,17598.352042,354.250,611.5625,895.6250,2302.3125,2127.3125,1251.1875,1.464486,3.764640,...,0.228411,2284112,0.1407,1407000,0.257472,2574722,0.440100,4401001,0.603767,6037665


In [16]:
a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 54 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   site       167 non-null    object 
 1   total_agb  167 non-null    float64
 2   psB1a      167 non-null    float64
 3   psB2a      167 non-null    float64
 4   psB3a      167 non-null    float64
 5   psB4a      167 non-null    float64
 6   psB5a      167 non-null    float64
 7   psB6a      167 non-null    float64
 8   ratio32fa  167 non-null    float64
 9   ratio42fa  167 non-null    float64
 10  ratio43fa  167 non-null    float64
 11  ratio52fa  167 non-null    float64
 12  ratio53fa  167 non-null    float64
 13  ratio54fa  167 non-null    float64
 14  ratio62fa  167 non-null    float64
 15  ratio63fa  167 non-null    float64
 16  ratio64fa  167 non-null    float64
 17  ratio65fa  167 non-null    float64
 18  ratio32a   167 non-null    int32  
 19  ratio42a   167 non-null    int32  
 20  ratio43a  

In [17]:
# # converts interger stage code dbg surface reflectance data for landsat 5 & 7 
# # to floating point for analysis of the vegetation index, blue band not used

# blued =((a['psB1d'].astype('int16')*0.0001)+0.0)

# greend =((a['psB2d'].astype('int16')*0.0001)+0.0)
# redd = ((a['psB3d'].astype('int16')*0.0001)+0.0)
# nird = ((a['psB4d'].astype('int16')*0.0001)+0.0)
# swir1d =((a['psB5d'].astype('int16')*0.0001)+0.0)
# swir2d =((a['psB6d'].astype('int16')*0.0001)+0.0)

In [18]:
# # calculate the band ratios

# a['ratio32fd'] = (a['psB3d'] / a['psB2d'])
# a['ratio42fd'] = (a['psB4d'] / a['psB2d'])
# a['ratio43fd'] = (a['psB4d'] / a['psB3d'])
# a['ratio52fd'] = (a['psB5d'] / a['psB2d'])
# a['ratio53fd'] = (a['psB5d'] / a['psB3d'])
# a['ratio54fd'] = (a['psB5d'] / a['psB4d'])
# a['ratio62fd'] = (a['psB6d'] / a['psB2d'])
# a['ratio63fd'] = (a['psB6d'] / a['psB3d'])
# a['ratio64fd'] = (a['psB6d'] / a['psB4d'])
# a['ratio65fd'] = (a['psB6d'] / a['psB5d'])

# # calculate the band ratios and convert them to int32 bit at 7 decimal places

# a['ratio32d'] = np.int32(np.around(a['ratio32fd'] * 10 ** 7))

# a['ratio42d'] = np.int32(np.around(a['ratio42fd'] * 10 ** 7))

# a['ratio43d'] = np.int32(np.around(a['ratio43fd'] * 10 ** 7))

# a['ratio52d'] = np.int32(np.around(a['ratio52fd'] * 10 ** 7))

# a['ratio53d'] = np.int32(np.around(a['ratio53fd'] * 10 ** 7))

# a['ratio54d'] = np.int32(np.around(a['ratio54fd'] * 10 ** 7))

# a['ratio62d'] = np.int32(np.around(a['ratio62fd'] * 10 ** 7))

# a['ratio63d'] = np.int32(np.around(a['ratio63fd'] * 10 ** 7))

# a['ratio64d'] = np.int32(np.around(a['ratio64fd'] * 10 ** 7))

# a['ratio65d'] = np.int32(np.around(a['ratio65fd'] * 10 ** 7))

In [19]:
# # calculate the vegetation indices
# a['GSAVIfd'] = ((nird-greend)/(nird+greend+0.5))*(1.5)
# a['GSAVId'] = np.int32(np.around(a['GSAVIfd'] * 10 ** 7))

# a['GNDVIfd'] = (nird-greend)/(nird+greend)
# a['GNDVId'] = np.int32(np.around(a['GNDVIfd'] * 10 ** 7))

# a['CVIfd']= (nird/greend)*(redd/greend)
# a['CVId'] = np.int32(np.around(a['CVIfd'] * 10 ** 7))

# a['NDGIfd']= (greend-redd)/(greend+redd)
# a['NDGId'] = np.int32(np.around(a['NDGIfd'] * 10 ** 7))

# a['RIfd']= (redd-greend)/(redd+greend)
# a['RId'] = np.int32(np.around(a['RIfd'] * 10 ** 7))

# a['NBRfd']= (nird-swir2d)/(nird+swir2d)
# a['NBRd'] = np.int32(np.around(a['NBRfd'] * 10 ** 7))

# a['NDIIfd']= (nird-swir1d)/(nird+swir1d)
# a['NDIId'] = np.int32(np.around(a['NDIIfd'] * 10 ** 7))

# a['GDVIfd']= (nird-greend)
# a['GDVId'] = np.int32(np.around(a['GDVIfd'] * 10 ** 7))

# a['MSAVIfd'] = (2 * nird + 1 - np.sqrt((np.power(((2*nird)+1) , 2)) - (8 * (nird-redd))))/2
# a['MSAVId'] = np.int32(np.around(a['MSAVIfd'] * 10 ** 7))

# a['DVIfd'] = (nird-redd)
# a['DVId'] = np.int32(np.around(a['DVIfd'] * 10 ** 7))

# a['SAVIfd']= ((nird-redd)/(nird+redd+0.5))*(1+0.5)
# a['SAVId'] = np.int32(np.around(a['SAVIfa'] * 10 ** 7))

# a['NDVIfd']= (nird-redd)/(nird+redd)
# a['NDVId'] = np.int32(np.around(a['NDVIfd'] * 10 ** 7))

# a['MSRfd']= (((nird/redd)-1)/((np.sqrt(nird/redd))+1))
# a['MSRd'] = np.int32(np.around(a['MSRfd'] * 10 ** 7))

In [20]:
print (list(a))

['site', 'total_agb', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 'ratio32fa', 'ratio42fa', 'ratio43fa', 'ratio52fa', 'ratio53fa', 'ratio54fa', 'ratio62fa', 'ratio63fa', 'ratio64fa', 'ratio65fa', 'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a', 'GSAVIfa', 'GSAVIa', 'GNDVIfa', 'GNDVIa', 'CVIfa', 'CVIa', 'NDGIfa', 'NDGIa', 'RIfa', 'RIa', 'NBRfa', 'NBRa', 'NDIIfa', 'NDIIa', 'GDVIfa', 'GDVIa', 'MSAVIfa', 'MSAVIa', 'DVIfa', 'DVIa', 'SAVIfa', 'SAVIa', 'NDVIfa', 'NDVIa', 'MSRfa', 'MSRa']


In [21]:
# tr = a[['site', 'x', 'y', 'chm', 'std_1', 'Min_1', 'Max_1', 'count_1', 'perc5_1', 'perc10_1', 'perc25_1', 'perc50_1', 
#         'perc75_1', 'perc80_1', 'perc95_1', 'perc99_1', 'cov', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 
#         'psB1d', 'psB2d', 'psB3d', 'psB4d', 'psB5d', 'psB6d','ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a',
#         'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a','GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 'NBRa', 
#         'NDIIa', 'GDVIa',  'MSAVIa', 'DVIa','SAVIa', 'NDVIa', 'MSRa', 'ratio32d', 'ratio42d', 'ratio43d', 'ratio52d', 
#         'ratio53d', 'ratio54d','ratio62d', 'ratio63d', 'ratio64d','ratio65d', 'GSAVId', 'GNDVId',  'CVId', 'NDGId', 
#         'RId',  'NBRd', 'NDIId',  'GDVId', 'MSAVId','DVId',  'SAVId', 'NDVId',  'MSRd','zone',]]

In [22]:
# remove fa
tr = a[['site', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 
        'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 
        'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 
        'NBRa', 'NDIIa', 'GDVIa',  'MSAVIa', 'DVIa',  'SAVIa', 'NDVIa',  'MSRa']]

In [23]:
print(list(tr))

['site', 'psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a', 'ratio32a', 'ratio42a', 'ratio43a', 'ratio52a', 'ratio53a', 'ratio54a', 'ratio62a', 'ratio63a', 'ratio64a', 'ratio65a', 'GSAVIa', 'GNDVIa', 'CVIa', 'NDGIa', 'RIa', 'NBRa', 'NDIIa', 'GDVIa', 'MSAVIa', 'DVIa', 'SAVIa', 'NDVIa', 'MSRa']


In [24]:
val = tr.sample(5)

In [25]:
val.shape

(5, 30)

In [26]:
val

,site,psB1a,psB2a,psB3a,psB4a,psB5a,psB6a,ratio32a,ratio42a,ratio43a,...,NDGIa,RIa,NBRa,NDIIa,GDVIa,MSAVIa,DVIa,SAVIa,NDVIa,MSRa
34,nt004,381.6500,482.3500,539.8000,1277.2500,1530.5000,1307.3000,11191044,26479735,23661541,...,-558276,558276,-116099,-901318,795000,1313057,738000,1624120,4063877,5392213
36,lim01,352.5625,565.9375,874.0000,2152.0000,2488.8750,1517.1250,15443401,38025400,24622426,...,-2147325,2147325,1730717,-724138,1587000,2093235,1278000,2388487,4223397,5691535
60,amg10,408.3750,687.8125,1009.1875,2499.9375,2723.4375,1507.9375,14672422,36346206,24771784,...,-1898585,1898585,2476286,-428954,1812000,2357502,1490000,2626939,4247434,5737565
17,ntagfu0009,730.1875,1355.7500,2044.7500,3039.8125,4014.1875,3276.5000,15082058,22421630,14866426,...,-2027067,2027067,-375297,-1382390,1684000,1351286,995000,1480214,1957505,2193402
130,el02,635.8000,1061.8500,1579.4500,2765.2000,3043.0000,1866.3000,14874511,26041343,17507360,...,-1962121,1962121,1941265,-478650,1704000,1717255,1186000,1903896,2730203,3232945


In [27]:
tr.to_csv(r'{0}:\cdu\data\zonal_stats\output\{1}\dja_dbi_dim_dis_dka_stc_start_indices.csv'.format(drive, date), index=False)

In [28]:
df_si = pd.merge(df, tr,  how='left', on=['site'])

In [29]:
df_si.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Columns: 190 entries, uid to MSRa
dtypes: float64(162), int32(23), int64(2), object(3)
memory usage: 234.2+ KB


In [30]:
print (list(df_si))

['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'geometry', 'b1_dja_min', 'b1_dja_max', 'b1_dja_mean', 'b1_dja_med', 'b1_dja_std', 'b1_dja_p25', 'b1_dja_p50', 'b1_dja_p75', 'b1_dja_p95', 'b1_dja_p99', 'b1_dbi_min', 'b1_dbi_max', 'b1_dbi_mean', 'b1_dbi_med', 'b1_dbi_std', 'b1_dbi_p25', 'b1_dbi_p50', 'b1_dbi_p75', 'b1_dbi_p95', 'b1_dbi_p99', 'b2_dbi_min', 'b2_dbi_max', 'b2_dbi_mean', 'b2_dbi_med', 'b2_dbi_std', 'b2_dbi_p25', 'b2_dbi_p50', 'b2_dbi_p75', 'b2_dbi_p95', 'b2_dbi_p99', 'b3_dbi_min', 'b3_dbi_max', 'b3_dbi_mean', 'b3_dbi_med', 'b3_dbi_std', 'b3_dbi_p25', 'b3_dbi_p50', 'b3_dbi_p75', 'b3_dbi_p95', 'b3_dbi_p99', 'b4_dbi_min', 'b4_dbi_max', 'b4_dbi_mean', 'b4_dbi_med', 'b4_dbi_std', 'b4_dbi_p25', 'b4_dbi_p50', 'b4_dbi_p75', 'b4_dbi_p95', 'b4_dbi_p99', 'b5

In [31]:
drop_list = ['psB1a', 'psB2a', 'psB3a', 'psB4a', 'psB5a', 'psB6a']
# Drop multiple columns
df_si.drop(drop_list, axis = 1, inplace=True)

In [32]:
dict_ = {
    'ratio32a':'ratio32m', 'ratio42a': 'ratio42m',
    'ratio43a': 'ratio43m', 'ratio52a': 'ratio52m',
    'ratio53a': 'ratio53m', 'ratio54a': 'ratio54m',
    'ratio62a': 'ratio62m', 'ratio63a': 'ratio63m',
    'ratio64a': 'ratio64m', 'ratio65a': 'ratio65m',
    'GSAVIa': 'GSAVIm', 'GNDVIa': 'GNDVIm',
    'CVIa': 'CVIm', 'NDGIa': 'NDGIm', 
    'RIa': 'RIm', 'NBRa': 'NBRm', 
    'NDIIa': 'NDIIm', 'GDVIa': 'GDVIm', 
    'MSAVIa': 'MSAVIm', 'DVIa': 'DVIm', 
    'SAVIa': 'SAVIm', 'NDVIa': 'NDVIm', 'MSRa': 'MSRm'
}

In [33]:
df_si.rename(columns=dict_, inplace=True)

In [34]:
df_si.to_csv(r'{0}:\cdu\data\zonal_stats\output\{1}\dja_dbi_dim_dis_dka_stc_h99a2_fpca2_dry_indicies.csv'.format(drive, date), index=False)

In [35]:
#E:\cdu\data\zonal_stats\output\dja_dbi_dim_dis_dka_stc_h99a2_fpca2_dry_clean_df.csv


In [36]:
def drop_cols_fn(df):
    
    drop_list = ['uid', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 
             'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha',
             'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'geometry']
    df.drop(drop_list, axis=1, inplace=True)
    
    return df

In [37]:
df_si_clean = drop_cols_fn(df_si)

In [38]:
df_si_clean.to_csv(r'{0}:\cdu\data\zonal_stats\output\{1}\dja_dbi_dim_dis_dka_stc_h99a2_fpca2_dry_indicies_clean.csv'.format(drive, date), index=False)